In [1]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 2.7 MB/s eta 0:00:00a 0:00:01


In [4]:
import requests
import json

def paraphrase_text_llama3(text: str):
    messages = [
        {"role": "user", "content": '''Reescreva todo o texto abaixo de forma que ele mantenha sua mensagem intacta, ou seja, o parafraseie, trocando palavras por sinonimos quando possivel deixando o texto de forma significantemente diferente com criatividade.
        
        A cidade se cobre com a escuridão noturna, e os edifícios brilham com luzes cintilantes. Histórias antigas sussurram pelas ruas desertas, enquanto o vento brinca com folhas secas. Na serenidade, a lua testemunha o adormecer do mundo, antecipando um novo dia repleto de oportunidades e enigmas a serem desvendados.
        '''},
        {"role": "assistant", "content": "À noite, a cidade se envolve em um manto escuro, iluminado pelas luzes cintilantes dos edifícios. Ruas desertas sussurram histórias antigas, enquanto o vento brinca com folhas secas dançando pelo asfalto. Na quietude, a lua observa o mundo adormecer, prometendo um novo amanhecer repleto de oportunidades e mistérios a desvendar sob seu suave brilho noturno."},
        {"role": "user", "content": f'''Reescreva todo o texto abaixo de forma que ele mantenha sua mensagem intacta, ou seja, o parafraseie, trocando palavras por sinonimos quando possivel deixando o texto de forma significantemente diferente com criatividade.
        
        {text}
        '''}
    ]
    
    
    return requests.post("http://localhost:11434/api/chat", json={
        "model": "llama3",
        "messages": messages,
        "stream": False,
        "options": {"temperature": 0.2}
    }).json()["message"]["content"]

In [5]:
paraphrase_text_llama3("ola eu sou goku")

'A luz do destino brilha intensamente, e eu, um guerreiro da justiça, surge como uma estrela no firmamento. Sou Goku, o defensor dos universos, pronto para enfrentar os desafios que se aproximam e proteger a harmonia cósmica.'

In [ ]:
curl -X POST http://localhost:11434/api/generate -d '{
  "model": "llama3",
  "prompt":"Why is the sky blue?",
 "stream": false}'

In [2]:
import pandas as pd
fake_train = pd.read_csv("./fake_train_balanced_indexed.csv")
fake_train

,Unnamed: 0,text,misinformation,index
0,2,EU GOSTARIA DE PEDIR A TODOS DO GRUPOS ENTREM ...,0,2
1,3,"O Brasil que eu quero, é um Brasil com mais FR...",0,3
2,4,Já que a GLOBO não mostra...\n\n*BOLSONARO HOJ...,0,4
3,5,Como são as coisas.\nChefe do jacaré aparece n...,1,5
4,7,Há informações fidedignas que o desmoralizado ...,1,7
...,...,...,...,...
13475,16770,Brasileiros comemoram aposentadoria expressa d...,1,16770
13476,16771,MST invade faixa olímpica e declara latifúndio...,1,16771
13477,16776,Lula assume Ministério da Defesa e promete tre...,1,16776
13478,16790,Pedreiro instala privadas como assentos em pon...,0,16790


In [3]:
pairs = [(x, x+100) if x + 100 < fake_train.shape[0] else (x, fake_train.shape[0])  for x in range(9900,fake_train.shape[0], 100)]
pairs

[(9900, 10000),
 (10000, 10100),
 (10100, 10200),
 (10200, 10300),
 (10300, 10400),
 (10400, 10500),
 (10500, 10600),
 (10600, 10700),
 (10700, 10800),
 (10800, 10900),
 (10900, 11000),
 (11000, 11100),
 (11100, 11200),
 (11200, 11300),
 (11300, 11400),
 (11400, 11500),
 (11500, 11600),
 (11600, 11700),
 (11700, 11800),
 (11800, 11900),
 (11900, 12000),
 (12000, 12100),
 (12100, 12200),
 (12200, 12300),
 (12300, 12400),
 (12400, 12500),
 (12500, 12600),
 (12600, 12700),
 (12700, 12800),
 (12800, 12900),
 (12900, 13000),
 (13000, 13100),
 (13100, 13200),
 (13200, 13300),
 (13300, 13400),
 (13400, 13480)]

In [6]:
import emoji
indexes = []
failed = []
for pair in pairs:
    paraphrased_text = pd.DataFrame({"text": [], "misinformation": []})
    for index, row in fake_train[pair[0]:pair[1]].iterrows():
        if len(row["text"].split()) <= 200:
            try:
                paraphrased = [
                    paraphrase_text_llama3(emoji.replace_emoji(row["text"]).encode("ascii", errors="ignore").decode()),
                    row["misinformation"]
                ]
                indexes.append(index)
                paraphrased_text.loc[paraphrased_text.shape[0]] = paraphrased
            except:
                try:
                    paraphrased = [
                        paraphrase_text_llama3(emoji.replace_emoji(row["text"]).encode("ascii", errors="ignore").decode()),
                        row["misinformation"]
                    ]
                    indexes.append(index)
                    paraphrased_text.loc[paraphrased_text.shape[0]] = paraphrased
                except:
                    try:
                        paraphrased = [
                            paraphrase_text_llama3(emoji.replace_emoji(row["text"]).encode("ascii", errors="ignore").decode()),
                            row["misinformation"]
                        ]
                        indexes.append(index)
                        paraphrased_text.loc[paraphrased_text.shape[0]] = paraphrased
                    except:
                        paraphrased = [
                            "FAILED",
                            -1
                        ]
                        indexes.append(index)
                        paraphrased_text.loc[paraphrased_text.shape[0]] = paraphrased
                        pass
    print(pair)
    paraphrased_text.to_csv(f"./fake_train_inc_{pair[0]}_{pair[1]}_paraphrased_aug_partial_llama3.csv")

(9900, 10000)
(10000, 10100)
(10100, 10200)
(10200, 10300)
(10300, 10400)
(10400, 10500)
(10500, 10600)
(10600, 10700)
(10700, 10800)
(10800, 10900)
(10900, 11000)
(11000, 11100)
(11100, 11200)
(11200, 11300)
(11300, 11400)
(11400, 11500)
(11500, 11600)
(11600, 11700)
(11700, 11800)
(11800, 11900)
(11900, 12000)
(12000, 12100)
(12100, 12200)
(12200, 12300)
(12300, 12400)
(12400, 12500)
(12500, 12600)
(12600, 12700)
(12700, 12800)
(12800, 12900)
(12900, 13000)
(13000, 13100)
(13100, 13200)
(13200, 13300)
(13300, 13400)
(13400, 13480)


In [12]:
pairs = [(x, x+100) if x + 100 < fake_train.shape[0] else (x, fake_train.shape[0])  for x in range(0,fake_train.shape[0], 100)]
indexes=[]
for pair in pairs:
    paraphrased_text = pd.DataFrame({"text": [], "misinformation": []})
    for index, row in fake_train[pair[0]:pair[1]].iterrows():
        if len(row["text"].split()) <= 200:
            indexes.append(row["index"])

In [10]:
#paraphrased_text.to_csv("./fake_train_inc_4000_n_paraphrased_aug.csv")

In [13]:
fake_full_paraprhased = pd.read_csv("./fake_train_inc_0_100_paraphrased_aug_partial_llama3.csv")
for pair in pairs[1:]:
    fake_full_paraprhased = pd.concat([fake_full_paraprhased,pd.read_csv(f"./fake_train_inc_{pair[0]}_{pair[1]}_paraphrased_aug_partial_llama3.csv")])

fake_full_paraprhased["index"] = indexes
    
fake_full_paraprhased

,Unnamed: 0,text,misinformation,index
0,0,Eu gostaria de pedir a todos os membros do gru...,0,2
1,1,"O país que sonho, um território onde a PAIXÃO ...",0,3
2,2,Aqui vai uma reescrita do texto:\n\nJá é sabid...,0,4
3,3,A vida tem seu jeito de se desenrolar.\n\nNo c...,1,5
4,4,"Dois gigantes do setor de consumo, Coca-Cola e...",0,8
...,...,...,...,...
15,15,O Espírito de Carnaval Seguro o Culto estreou ...,1,16758
16,16,A equipe da Polícia Federal realizou uma busca...,1,16764
17,17,"Semelhante a uma página em branco, Geraldo Alc...",1,16765
18,18,"A ocupação pacífica da ""Faixa Olímpica"" pela c...",1,16771


In [15]:
fake_full_paraprhased[fake_full_paraprhased["misinformation"] != -1].to_csv("fake_train_full_paraphrased_aug_indexed_balanced_llama3.csv")

In [6]:
fake_partial = pd.read_csv("./fake_train_inc_0_100_paraphrased_aug_partial.csv")
for pair in pairs[1:]:
    fake_partial = pd.concat([fake_partial,pd.read_csv(f"./fake_train_inc_{pair[0]}_{pair[1]}_paraphrased_aug_partial.csv")])

fake_partial["index"] = indexes
    
fake_partial

FileNotFoundError: [Errno 2] No such file or directory: './fake_train_inc_4000_4100_paraphrased_aug_partial.csv'

In [18]:
import pandas as pd
fake_train_full_p_aug = pd.read_csv("./fake_train_inc_full_paraphrased_aug_indexed_useful_no_error.csv").iloc[:, -3:]
fake_train_full_p_aug = pd.concat([fake_train_full_p_aug, fake_partial.iloc[:, -3:]])

In [25]:
fake_train_full_p_aug = fake_train_full_p_aug.sort_values(by=['index'])
fake_train_full_p_aug[fake_train_full_p_aug["misinformation"] != -1].to_csv("fake_train_inc_full_paraphrased_aug_indexed_balanced_no_errors_v1.csv")

In [27]:
fake_train_full_p_aug[fake_train_full_p_aug["misinformation"] != -1]

,text,misinformation,index
0,Desejo solicitar a todos os membros deste grup...,0,2
1,"A visão do Brasil que desejo, é um país cheio ...",0,3
2,"Quanto ao texto abaixo, ele foi reescrito de f...",0,4
3,Está tudo bem? O líder do grupo aparece em um ...,1,5
4,"As duas principais forças de vendas mundiais, ...",0,8
...,...,...,...
69,"O bloco de rua ""Segura o Culto"" estreou em Oli...",1,16758
70,A Polícia Federal procurou Lula no suposto sít...,1,16764
71,"Branco e nulo, Alckmin celebra resultados do D...",1,16765
72,O Movimento dos Sem Terra (MST) invade uma áre...,1,16771
